In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import array_intersect

# Iniciar uma sessão Spark
spark = SparkSession.builder \
    .appName("Read Parquet File") \
    .getOrCreate()

# Substitua 'caminho/do/arquivo.parquet' pelo caminho real do seu arquivo .parquet
file_path1 = 'part-00000-b5473193-ba27-4dc1-aa69-ec770eb7ef82-c000.snappy.parquet'

file_path2 = 'part-00000-f43e1eee-9720-4ebb-947c-626c9e1ba2f1-c000.snappy.parquet'

# Carregar o arquivo .parquet para um DataFrame
json = spark.read.parquet(file_path1)
csv = spark.read.parquet(file_path2)

csv = csv.filter(csv["genero"].contains("Drama") & (csv["anoLancamento"] >= 2000))

# Extrair os nomes dos gêneros como uma lista de strings
json = json.withColumn("genre_names", expr("transform(genres, x -> x.name)"))

# Filtrar os filmes de drama desde 2000 no DataFrame json
json = json.filter(array_contains(json["genre_names"], "Drama") & (year("release_date") >= 2000))

joined_df = json.join(csv, json["imdb_id"] == csv["id"], "inner")

# Seleciona as colunas necessárias para os diferentes datasets
# Selecionando as colunas relevantes para a dimensão de filmes
filme_dim = joined_df.select(
    json["imdb_id"].alias("id"),
    csv["tituloPrincipal"].alias("tituloCSV"),
    json["title"].alias("tituloJSON"),
    json["release_date"].alias("lancamento"),
    json["original_language"].alias("idioma"),
    json["overview"].alias("resumo"),
)

# Selecionando as colunas relevantes para a fato de filmes
filme_fato = joined_df.select(
    json["imdb_id"].alias("id"),
    csv["notaMedia"].alias("notaIMDB"),
    csv["numeroVotos"].alias("votosIMDB"),
    json["vote_average"].alias("notaTMDB"),
    json["vote_count"].alias("votosTMDB"),
    json["popularity"].alias("popularidade"),
    json["budget"].alias("orcamento"),
    json["revenue"].alias("receita"),
)

# Explode os países de produção para uma dimensão de países
paises = json.select(explode("production_countries").alias("origem"))
paises_dim = paises.selectExpr(
    "origem.iso_3166_1 as codigo",
    "origem.name as nome"
).distinct()

# Cria uma associação entre filmes e países
paises_exploded = json.select("imdb_id", explode("production_countries.iso_3166_1").alias("codigo_origem"))
associacao_filme_pais = paises_exploded.join(paises_dim, paises_exploded["codigo_origem"] == paises_dim["codigo"], "inner").select("imdb_id", "codigo")
associacao_filme_pais = associacao_filme_pais.withColumnRenamed("imdb_id", "id")


filme_fato.show()
filme_dim.show()
paises_dim.show()
associacao_filme_pais.show()


# Finalizar a sessão Spark
spark.stop()


24/04/09 09:14:40 WARN Utils: Your hostname, Super-Mac-n-cheese.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en1)
24/04/09 09:14:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 09:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+--------+---------+--------+---------+------------+---------+---------+
|       id|notaIMDB|votosIMDB|notaTMDB|votosTMDB|popularidade|orcamento|  receita|
+---------+--------+---------+--------+---------+------------+---------+---------+
|tt0116391|     6.2|      254|     5.0|        2|       1.079|        0|        0|
|tt0116748|     3.8|      273|     7.0|        3|        1.07|        0|        0|
|tt0118589|     2.4|    23624|     4.4|      133|       8.525| 22000000|  5271666|
|tt0118694|     8.1|   150711|    8.11|     2516|      45.566|   150000| 14204632|
|tt0118926|     6.9|     6683|     6.1|       54|       8.869|        0|  5227348|
|tt0120467|     5.2|     4158|     5.4|       46|       4.658|   120000|    14904|
|tt0138353|     5.7|      645|     4.4|        5|       1.468|  1700000|        0|
|tt0138946|     6.4|    21799|   6.178|      414|      13.811|        0|        0|
|tt0140806|     6.8|      104|     0.0|        0|       1.099|        0|        0|
|tt0